In [ ]:
import pandas as pd

# Dialogue set from Cornell Movie-Dialogs Corpus

# Lines tsv
lines_raw = pd.read_csv("dialogue/movie_lines.tsv", header=None, names=["raw"])
lines_split = lines_raw["raw"].str.split("\t", expand=False)

fixed_rows = []
for row in lines_split:
    if len(row) > 5:
        row = row[:4] + [" ".join(row[4:])]
    elif len(row) < 5:
        row += [None] * (5 - len(row))
    fixed_rows.append(row)

lines = pd.DataFrame(fixed_rows, columns=["lineID", "characterID", "movieID", "characterName", "text"])

# Characters tsv
characters_raw = pd.read_csv("dialogue/movie_characters_metadata.tsv", header=None, names=["raw"])
characters_split = characters_raw["raw"].str.split("\t", expand=False)

fixed_rows = []
for row in characters_split:
    if len(row) > 6:
        row = row[:4] + [" ".join(row[4:])]
    elif len(row) < 6:
        row += [None] * (6 - len(row))
    fixed_rows.append(row)

characters = pd.DataFrame(fixed_rows, columns=["characterID", "characterName", "movieID", "movieTitle", "gender", "creditPosition"])

# movies tsv
movies = pd.read_csv("dialogue/movie_titles_metadata.tsv", header=None, delimiter="\t", names=["movieID", "title", "year", "imdbRating", "imdbVotes", "genres"])

In [26]:
# The Pudding analysis complete gender data:
gender_data = pd.read_csv("pudding/character_list5.csv", header=0, encoding="latin-1")
script_metadata = pd.read_csv("pudding/meta_data7.csv", header=0, encoding="latin-1")

In [ ]:
# Using NLTK to tokenize text
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
lines["tokens"] = lines["text"].apply(lambda x: word_tokenize(x) if pd.notnull(x) else [])
lines["token_count"] = lines["tokens"].apply(len)
lines["word_count"] = lines["text"].apply(lambda x: len(x.split()) if pd.notnull(x) else 0)